# 1. Import Packages for the Environment

In [1]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
IN_KAGGLE = os.path.exists('/kaggle/input')
IN_COLAB = not IN_KAGGLE and os.path.exists('/content')
IN_DEIB = not IN_KAGGLE and not IN_COLAB

In [4]:
if not IN_DEIB:
    !pip install nnunetv2
    !pip install captum

# 2. Mount the dataset

In [5]:
from batchgenerators.utilities.file_and_folder_operations import join

if IN_COLAB:
    # Google Colab
    # for colab users only - mounting the drive

    from google.colab import drive
    drive.mount('/content/drive',force_remount = True)

    drive_dir = "/content/drive/My Drive"
    mount_dir = join(drive_dir, "tesi", "automi")
    base_dir = os.getcwd()
elif IN_KAGGLE:
    # Kaggle
    mount_dir = "/kaggle/input/automi-seg"
    base_dir = os.getcwd()
    print(base_dir)
    !ls '/kaggle/input'
    !cd "/kaggle/input/automi-seg" ; ls
else:
    mount_dir = "/workspace/data"
    base_dir = "/workspace/output"
    os.chdir(base_dir)
    print("base_dir:", base_dir)

base_dir: /workspace/output


# 3. Setting up nnU-Nets folder structure and environment variables
nnUnet expects a certain folder structure and environment variables.

Roughly they tell nnUnet:
1. Where to look for stuff
2. Where to put stuff

For more information about this please check: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md

## 3.1 Set environment Variables and creating folders

In [6]:
# ===========================
# 📦 SETUP nnUNet ENVIRONMENT
# ===========================

# Definisci i path da settare
path_dict = {
    "nnUNet_raw": join(mount_dir, "nnunet_raw"),
    "nnUNet_preprocessed": join(mount_dir, "preprocessed_files"),#"nnUNet_preprocessed"),
    "nnUNet_results": join(mount_dir, "results"),#"nnUNet_results"),
    # "RAW_DATA_PATH": join(mount_dir, "RawData"),  # Facoltativo, se ti serve salvare zips
}

# Scrivi i path nelle variabili di ambiente, che vengono lette dal modulo paths di nnunetv2
for env_var, path in path_dict.items():
    os.environ[env_var] = path

from nnunetv2.paths import nnUNet_results, nnUNet_raw

if IN_KAGGLE:
    if nnUNet_raw == None:
        nnUNet_raw = "/kaggle/input/nnunet_raw"
    if nnUNet_results == None:
        nnUNet_results = "/kaggle/input/results"
    # Kaggle has some very unconsistent behaviors in dataset mounting...
    #nnUNet_raw = "/kaggle/input/automi-seg/nnunet_raw"
    #nnUNet_results = "/kaggle/input/automi-seg/results"
    
print("nnUNet_raw:", nnUNet_raw)
print("nnUNet_results:", nnUNet_results)

nnUNet_raw: /workspace/data/nnunet_raw
nnUNet_results: /workspace/data/results


### Some tests

In [7]:
# all volumes of fold 0 test set
volume_codes = ["00004", "00005", "00024", "00027", "00029", "00034", "00039", "00044"]

In [8]:
ct_img_paths = {}
organ_map_paths = {}

for volume_code in volume_codes:
    if IN_KAGGLE:
        ct_img_paths[volume_code] = join(nnUNet_raw, "imagesTr", f"AUTOMI_{volume_code}_0000.nii")
        organ_map_paths[volume_code] = join(nnUNet_raw, "total_segmentator_structures", f"AUTOMI_{volume_code}_0000", "mask_mask_add_input_20_total_segmentator.nii")
    else:
        ct_img_paths[volume_code] = join(nnUNet_raw, "imagesTr", f"AUTOMI_{volume_code}_0000.nii.gz")
        organ_map_paths[volume_code] = join(nnUNet_raw, "total_segmentator_structures", f"AUTOMI_{volume_code}_0000", "mask_mask_add_input_20_total_segmentator.nii.gz")
    ct_img = nib.load(ct_img_paths[volume_code])
    organ_map = nib.load(organ_map_paths[volume_code])
    print(f"Volume {volume_code}:")
    print("CT shape:", ct_img.shape)
    print("Organ shape:", organ_map.shape)
    print("Spacing:", ct_img.header.get_zooms())
    print("Organ spacing:", organ_map.header.get_zooms())
    assert np.all(ct_img.affine == organ_map.affine), "CT and organ mask affine matrices do not match!"

Volume 00004:
CT shape: (512, 512, 221)
Organ shape: (512, 512, 221)
Spacing: (1.3671875, 1.3671875, 5.0)
Organ spacing: (1.3671875, 1.3671875, 5.0)
Volume 00005:
CT shape: (512, 512, 219)
Organ shape: (512, 512, 219)
Spacing: (1.171875, 1.171875, 5.0)
Organ spacing: (1.171875, 1.171875, 5.0)
Volume 00024:
CT shape: (512, 512, 321)
Organ shape: (512, 512, 321)
Spacing: (0.976562, 0.976562, 3.0)
Organ spacing: (0.976562, 0.976562, 3.0)
Volume 00027:
CT shape: (512, 512, 236)
Organ shape: (512, 512, 236)
Spacing: (1.3671875, 1.3671875, 5.0)
Organ spacing: (1.3671875, 1.3671875, 5.0)
Volume 00029:
CT shape: (512, 512, 259)
Organ shape: (512, 512, 259)
Spacing: (1.367188, 1.367188, 7.5)
Organ spacing: (1.367188, 1.367188, 7.5)
Volume 00034:
CT shape: (512, 512, 249)
Organ shape: (512, 512, 249)
Spacing: (1.171875, 1.171875, 5.0)
Organ spacing: (1.171875, 1.171875, 5.0)
Volume 00039:
CT shape: (512, 512, 283)
Organ shape: (512, 512, 283)
Spacing: (1.171875, 1.171875, 5.0)
Organ spacing: (1.

In [9]:
ct_img = nib.load(ct_img_paths[volume_codes[1]])
print("affine:", ct_img.affine)

affine: [[-1.17187500e+00  0.00000000e+00  0.00000000e+00  3.00000000e+02]
 [ 0.00000000e+00 -1.17187500e+00  0.00000000e+00  1.85300003e+02]
 [ 0.00000000e+00  0.00000000e+00  5.00000000e+00 -1.43419995e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [10]:
from typing import Union

# define an utility for Nifty file saving
def save_nifty(data: Union[np.ndarray, torch.Tensor], affine, path, dtype=np.float32):
    if isinstance(data, torch.Tensor):
        data = data.detach().cpu().numpy()
    # set fixed type float32
    data = data.astype(dtype)
    nib.save(nib.Nifti1Image(data, affine), path)

def save_nifty_binary(data: Union[np.ndarray, torch.Tensor], affine, path):
    return save_nifty(data, affine, path, dtype=np.uint8)


# define an utility for annoying nnunetv2 preprocessing
def nnunetv2_default_preprocessing(ct_img_path, 
                                   predictor, 
                                   dataset_json_path,
                                   other_volumes: Union[np.ndarray, None] = None) -> np.ndarray:
    """
    Preprocesses the CT image and other volumes using nnunetv2's default preprocessing
    pipeline. This function reads the CT image, applies the preprocessor, and returns
    the preprocessed image.
    """
    plans_manager = predictor.plans_manager
    configuration_manager = predictor.configuration_manager
    
    preprocessor = configuration_manager.preprocessor_class(verbose=False)
    rw = plans_manager.image_reader_writer_class()
    if callable(rw) and not hasattr(rw, "read_images"):
        rw = rw()
    img_np, img_props = rw.read_images([str(ct_img_path)])
    
    preprocessed, other_volumes_preprocessed, _ = preprocessor.run_case_npy(
        img_np, seg=other_volumes, properties=img_props,
        plans_manager=plans_manager,
        configuration_manager=configuration_manager,
        dataset_json=dataset_json_path
    )
    if other_volumes:
        return preprocessed, other_volumes_preprocessed
    return preprocessed

## Evaluate attribution maps using ABPC (Area Between Perturbation Curves)

### Let's start with FCC supervoxel attribution maps, True Positive aggregation 

In [11]:
from pathlib import Path

attribution_map_paths = {}
for volume_code in volume_codes:
    attribution_map_paths[volume_code] = Path(join(volume_code, "attribution_map.nii.gz"))

In [ ]:
# define the perturbation function
def perturb(original_volume: torch.Tensor,
             interpretable_input: np.ndarray,
             feature_map: torch.Tensor) -> torch.Tensor:
    """"
    return the perturbed input and the perturbation map
    """
    # apply the perturbation to the original volume
    perturbed_volume = original_volume.clone()
    perturbed_volume[feature_map > 0] = interpretable_input[feature_map > 0]
    return perturbed_volume, perturbation_map

In [ ]:
from typing import Callable

# let's define the main block to compute one evaluation step, given the
# original volume, the interpretable input vector, and the forward function
def evaluate_step(original_volume: torch.Tensor,
                  interpretable_input: np.ndarray,
                  feature_map: torch.Tensor,
                  forward_function: Callable):
    # obtain perturbed output
    perturbed_output, perturbation_map = perturb(original_volume, 
                                                 interpretable_input, 
                                                 feature_map)

    # compute the forward pass with the perturbed input
    output = forward_function(perturbed_output, perturbation_map)

    return output
